# Modèle V1.4 de voiture autonome sur DonkeyCarSimulator

In [1]:
import os
import numpy as np
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from time import time
from tqdm import tqdm
import json
from PIL import Image
import base64
from io import BytesIO
import inspect

In [2]:
TIME=str(time())

MODEL_NAME = "DCModelV1.4-renault-"+TIME

STORAGE_ROOT_DIR = "/home/nigiva/git/lopilo-trainer/data"

DATASET_NAME = "renault_merged_58800"
DATASET_LABEL_FILENAME = "label.csv"

DATASET_PATH = os.path.join(STORAGE_ROOT_DIR, "sample", "extern", DATASET_NAME)
DATASET_LABEL_PATH = os.path.join(DATASET_PATH, DATASET_LABEL_FILENAME)
TENSORLOG_PATH = os.path.join(STORAGE_ROOT_DIR, "log", "extern", MODEL_NAME)
os.mkdir(TENSORLOG_PATH)

BATCH_SIZE = 64

SAVE_PATH = os.path.join(STORAGE_ROOT_DIR, "model", "extern", MODEL_NAME)
IMAGE_SHAPE = (120,160, 1)
print("Ok.")

Ok.


# 0. Extraire la dataset
## A) zip en dataset

In [14]:
!rm -Rf "/home/nigiva/git/dkarstream/Samples/3_controller_generated_track_0_48000"
!unzip -q "/home/nigiva/git/dkarstream/Samples/3_controller_generated_track_0_48000.zip" -d "/home/nigiva/git/dkarstream/Samples/"
!ls "/home/nigiva/git/dkarstream/Samples"

3_controller_generated_track_0_48000  3_controller_generated_track_0_48000.zip


## B) json (.eslr) en dataset
On convertit ligne par ligne les données envoyées par le serveur en :
- une image
- une ligne dans le csv label.csv

In [6]:
!rm -Rf "/home/nigiva/git/lopilo-trainer/data/sample/extern/renault_merged_58800"

In [7]:
DATASET_JSON_FILE = DATASET_PATH + ".eslr"

IMAGE_PATH = "images"
DATASET_IMAGE_PATH = os.path.join(DATASET_PATH, IMAGE_PATH)

if not os.path.exists(DATASET_PATH):
    os.mkdir(DATASET_PATH)
if not os.path.exists(DATASET_IMAGE_PATH):
    os.mkdir(DATASET_IMAGE_PATH)

In [8]:
DATA_LABEL_FILE = open(DATASET_LABEL_PATH, "w")
DATA_LABEL_FILE.write("path,angle,throttle\n")
with open(DATASET_JSON_FILE, "r") as dataset_file:
    for i, line in enumerate(tqdm(dataset_file)):
        data_line = json.loads(line)
        if (data_line["msg_type"] == "telemetry"):
            image_relative_path = os.path.join(IMAGE_PATH, str(i)+".jpg")
            image_absolute_path = os.path.join(DATASET_PATH, image_relative_path)
            Image.open(BytesIO(base64.b64decode(data_line["image"]))).save(image_absolute_path)
            # Toutes les données à ajouter dans la dataset
            data2write = [image_relative_path, str(data_line["steering_angle"]), str(data_line["throttle"])]
            DATA_LABEL_FILE.write(",".join(data2write) + "\n")
DATA_LABEL_FILE.close()
print("ok.")

58821it [01:13, 804.03it/s]

ok.


## 1. Préparer la dataset

In [9]:
# Obetnir les valeurs du fichier contenant les labels
raw_data = pd.read_csv(DATASET_LABEL_PATH)

# change la chemin de fichier
def change_path(path):
  return os.path.join(DATASET_PATH, path)
raw_data['path'] = raw_data['path'].map(change_path)
print(raw_data)

                                                    path     angle  throttle
0      /home/nigiva/git/lopilo-trainer/data/sample/ex...  0.098022       0.2
1      /home/nigiva/git/lopilo-trainer/data/sample/ex... -0.239227       0.2
2      /home/nigiva/git/lopilo-trainer/data/sample/ex... -0.239227       0.2
3      /home/nigiva/git/lopilo-trainer/data/sample/ex... -0.239227       0.2
4      /home/nigiva/git/lopilo-trainer/data/sample/ex... -0.364716       0.2
...                                                  ...       ...       ...
58815  /home/nigiva/git/lopilo-trainer/data/sample/ex...  0.152924       0.2
58816  /home/nigiva/git/lopilo-trainer/data/sample/ex...  0.160767       0.2
58817  /home/nigiva/git/lopilo-trainer/data/sample/ex...  0.160767       0.2
58818  /home/nigiva/git/lopilo-trainer/data/sample/ex...  0.160767       0.2
58819  /home/nigiva/git/lopilo-trainer/data/sample/ex...  0.160767       0.2

[58820 rows x 3 columns]


### Split en 3 jeux : Train, Test et Validation

In [10]:
train_and_test_set, validation_set = train_test_split(raw_data,
                                             test_size = 0.15,
                                             shuffle = True)
train_set, test_set = train_test_split(train_and_test_set,
                                             test_size = 0.20,
                                             shuffle = True)
NBR_ROW_TRAIN_SET = train_set.shape[0]
NBR_ROW_TEST_SET = test_set.shape[0]
NBR_ROW_VALIDATION_SET = validation_set.shape[0]
print(train_set)
print(test_set)
print(validation_set)

                                                    path     angle  throttle
31557  /home/nigiva/git/lopilo-trainer/data/sample/ex...  0.160767       0.2
42010  /home/nigiva/git/lopilo-trainer/data/sample/ex...  0.000000       0.2
3662   /home/nigiva/git/lopilo-trainer/data/sample/ex... -0.239227       0.2
5458   /home/nigiva/git/lopilo-trainer/data/sample/ex...  0.000000       0.2
2493   /home/nigiva/git/lopilo-trainer/data/sample/ex... -0.098053       0.2
...                                                  ...       ...       ...
43508  /home/nigiva/git/lopilo-trainer/data/sample/ex...  0.000000       0.2
38484  /home/nigiva/git/lopilo-trainer/data/sample/ex...  0.419586       0.2
18203  /home/nigiva/git/lopilo-trainer/data/sample/ex...  0.129394       0.2
52707  /home/nigiva/git/lopilo-trainer/data/sample/ex...  0.505859       0.2
34231  /home/nigiva/git/lopilo-trainer/data/sample/ex...  0.000000       0.2

[39997 rows x 3 columns]
                                                  

### Traitements avec TensorData

In [11]:
# Mettre dans des tensors
train_tensor = tf.data.Dataset.from_tensor_slices(({"input" : train_set['path']}, {"angle" : train_set['angle'], "throttle" : train_set['throttle']}))
test_tensor = tf.data.Dataset.from_tensor_slices(({"input" : test_set['path']}, {"angle" : test_set['angle'], "throttle" : test_set['throttle']}))
validation_tensor = tf.data.Dataset.from_tensor_slices(({"input" : validation_set['path']}, {"angle" : validation_set['angle'], "throttle" : validation_set['throttle']}))

# Definir les fonctions de chargement des images et de mapping
def load_and_preprocess_image(path):
    file_content = tf.io.read_file(path['input'])
    tricolors_img = tf.cast(tf.image.decode_jpeg(file_content, channels=3), dtype=tf.float32)
    gray_img = tf.image.rgb_to_grayscale(tricolors_img)
    # FIX change / 255.0 by / 255.0  -  0.5, to have a range between -0.5 et 0.5 (instead of 0 and 1)
    normalized_img = (gray_img / 127.5) - 1
    normalized_img = tf.reshape(normalized_img, IMAGE_SHAPE)
    print(normalized_img)
    return {"input": normalized_img}

def load_map_function(path, d):
    return load_and_preprocess_image(path), d

# Appliquer le mapping aux tensors
train_tensor_normalized = train_tensor.map(load_map_function, num_parallel_calls=3)
test_tensor_normalized = test_tensor.map(load_map_function, num_parallel_calls=3)
validation_tensor_normalized = validation_tensor.map(load_map_function, num_parallel_calls=3)

Tensor("Reshape:0", shape=(120, 160, 1), dtype=float32)
Tensor("Reshape:0", shape=(120, 160, 1), dtype=float32)
Tensor("Reshape:0", shape=(120, 160, 1), dtype=float32)


In [12]:
# On mélange les datasets (on fixe un nombre d'exemple tiré au sort, ici 20 000 exemples sur les 30 000)
# On demande un prechargement à l'avance de toujours 3 exemples
train_dataset = train_tensor_normalized.shuffle(20000).batch(BATCH_SIZE).prefetch(2)
test_dataset = test_tensor_normalized.shuffle(8000).batch(BATCH_SIZE).prefetch(2)
validation_dataset = validation_tensor_normalized.shuffle(8000).batch(BATCH_SIZE).prefetch(2)

## 2. Le modèle

In [30]:
class ModelSaver:
  """
  Save the model into a file ('model.code') as source code
  """
  def __init__(self, activate = True):
    """
    Initialize the saver
    :param activate: capture or not the source code of decorated functions
    """
    self.is_activated = activate
    self.s_init = None
    self.s_call = None
  def init(self, funct):
    """
    Capture the source code of the init function
    ---
    Use as a decoration
    Such as :
    ```
    MODEL_SAVER = ModelSaver(True)
    ...
    @MODEL_SAVER.init
    def __init__(self, name = ""):
      ...
    ```
    """
    def funct_with_params(*args, **kwargs):
      return funct(*args, **kwargs)
    if self.is_activated:
      self.s_init = inspect.getsource(funct)
    return funct_with_params

  def call(self, funct):
    """
    Capture the source code of the init function
    ---
    Use as a decoration
    Such as :
    ```
        MODEL_SAVER = ModelSaver(True)
        ...
        @MODEL_SAVER.call
        def call(self):
            ...
    ```
    """
    def funct_with_params(*args, **kwargs):
      return funct(*args, **kwargs)
    if self.is_activated:
      self.s_call = inspect.getsource(funct)
    return funct_with_params
  
  def save(self, path):
    """
    Save the source code of the model as a file
    :param path: file path
    """
    if self.s_init is not None and self.s_call is not None:
      with open(path, "w") as s:
        s.write("class DCModel(keras.Model):\n")
        s.write("  MODEL_SAVER = ModelSaver(False)\n")
        s.write(self.s_init)
        s.write(self.s_call)
    else:
      raise Exception("init or call function are not saved")
  @staticmethod
  def load(path):
    """
    Load the Model source code
    :param path: file path
    """
    with open(path, "r") as s:
      exec(s.read())

In [31]:
LR = 0.001

In [36]:
MODEL_SAVER = ModelSaver(True)

class DCModel(keras.Model):
  @MODEL_SAVER.init
  def __init__(self, name="DCModel"):
    super(DCModel, self).__init__(name=name)
    #self.input_layer = keras.layers.Input(shape=IMAGE_SHAPE, name='input')
    self.cnn_1 = keras.layers.Conv2D(12, (5, 5), strides=(2, 2), padding="same", kernel_initializer='he_uniform', activation='relu', name='input')
    self.bn_1 = tf.keras.layers.BatchNormalization()

    self.cnn_2 = keras.layers.Conv2D(16, (5, 5), strides=(2, 2), padding="same", kernel_initializer='he_uniform', activation='relu')
    self.bn_2 = tf.keras.layers.BatchNormalization()

    self.cnn_3 = keras.layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same", kernel_initializer='he_uniform', activation='relu')
    self.bn_3 = tf.keras.layers.BatchNormalization()

    self.cnn_4 = keras.layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same", kernel_initializer='he_uniform', activation='relu')
    self.bn_4 = tf.keras.layers.BatchNormalization()

    self.cnn_5 = keras.layers.Conv2D(48, (3, 3), strides=(2, 2), padding="same", kernel_initializer='he_uniform', activation='relu')
    self.bn_5 = tf.keras.layers.BatchNormalization()

    self.drop_1 = keras.layers.Dropout(0.2)

    self.flat = keras.layers.Flatten()

    self.dense_1 = keras.layers.Dense(100, kernel_initializer='he_uniform', activation='elu')
    self.bn_6 = tf.keras.layers.BatchNormalization()
    self.drop_2 = keras.layers.Dropout(0.1)

    self.dense_2 = keras.layers.Dense(50, kernel_initializer='he_uniform', activation='elu')
    self.bn_7 = tf.keras.layers.BatchNormalization()
    self.drop_3 = keras.layers.Dropout(0.1)

    self.dense_3 = keras.layers.Dense(25, kernel_initializer='he_uniform', activation='elu')
    self.bn_8 = tf.keras.layers.BatchNormalization()
    self.drop_4 = keras.layers.Dropout(0.1)

    self.dense_4 = keras.layers.Dense(25, kernel_initializer='he_uniform', activation='elu')
    self.bn_9 = tf.keras.layers.BatchNormalization()

    self.output_layer_1 = keras.layers.Dense(1, activation='linear', name='angle')
    self.output_layer_2 = keras.layers.Dense(1, activation='linear', name='throttle')

  
  @MODEL_SAVER.call
  @tf.function
  def call(self, inputs, training=False):
    l = self.cnn_1(inputs['input'])
    l = self.bn_1(l)
    
    l = self.cnn_2(l)
    l = self.bn_2(l)

    l = self.cnn_3(l)
    l = self.bn_3(l)

    l = self.cnn_4(l)
    l = self.bn_4(l)

    l = self.cnn_5(l)
    l = self.bn_5(l)

    l = self.drop_1(l)

    l = self.flat(l)

    l = self.dense_1(l)
    l = self.bn_6(l)
    l = self.drop_2(l)

    l = self.dense_2(l)
    l = self.bn_7(l)
    l = self.drop_3(l)

    l = self.dense_3(l)
    l = self.bn_8(l)
    l = self.drop_4(l)

    l = self.dense_4(l)
    l = self.bn_9(l)
    
    return {'angle' : self.output_layer_1(l), 'throttle' : self.output_layer_2(l)}

model = DCModel(name='DonkeyCarModel')

optimizer = keras.optimizers.Adam(learning_rate=LR)
model.compile(optimizer=optimizer,loss=keras.losses.MSE, metrics=["mse"])

## 3. L'entrainement

In [37]:
#@title Les hyperparamètres de l'entrainement
INITIAL_EPOQUE = 0
NBR_EPOQUES = 1
NBR_EPOQUES_APRES_EARLY_STOPPING = 4

In [12]:
# Définir mes callback
cb = [
    keras.callbacks.EarlyStopping(patience=NBR_EPOQUES_APRES_EARLY_STOPPING,
                                  restore_best_weights=True),
    keras.callbacks.TensorBoard(log_dir=TENSORLOG_PATH)
    ]

# Fit mon modèle
model.fit(train_dataset,
          validation_data=test_dataset,
          epochs=NBR_EPOQUES,
          initial_epoch = INITIAL_EPOQUE,
          callbacks=cb)

625/625 [==============================] - 151s 236ms/step - loss: 0.0443 - angle_loss: 0.0364 - throttle_loss: 0.0080 - angle_mse: 0.0364 - throttle_mse: 0.0080 - val_loss: 0.0128 - val_angle_loss: 0.0113 - val_throttle_loss: 0.0015 - val_angle_mse: 0.0113 - val_throttle_mse: 0.0015


In [13]:
#model.save(SAVE_PATH, save_format='tf')
model.save_weights(os.path.join(SAVE_PATH, "weights.data"))
MODEL_SAVER.save(os.path.join(SAVE_PATH, "model.code"))

In [14]:
print(MODEL_NAME)

DCModelV1.4-renault-1614446893.405174


In [15]:
model.summary()

Model: "DonkeyCarModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (Conv2D)               multiple                  312       
_________________________________________________________________
batch_normalization (BatchNo multiple                  48        
_________________________________________________________________
conv2d (Conv2D)              multiple                  4816      
_________________________________________________________________
batch_normalization_1 (Batch multiple                  64        
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  4640      
_________________________________________________________________
batch_normalization_2 (Batch multiple                  128       
_________________________________________________________________
conv2d_2 (Conv2D)            multiple               

In [50]:
performances = model.evaluate(validation_dataset, batch_size=32)
print("Loss Angle :", performances[0],"Loss Throttle :", performances[1],"Acc Angle :", performances[2],"Acc Throttle :", performances[3])

48/48 [==============================] - 5s 86ms/step - loss: 0.0723 - angle_loss: 0.0419 - throttle_loss: 0.0304 - angle_mse: 0.0419 - throttle_mse: 0.0304
Loss Angle : 0.07233501225709915 Loss Throttle : 0.041900672018527985 Acc Angle : 0.030434345826506615 Acc Throttle : 0.041900672018527985
